In [1]:
import os
os.chdir('..')
import sqlite3
from dotenv import load_dotenv
from selenium import webdriver
from helper import set_filters_huurstunt
from helper_database import should_send_email, mark_email_as_sent
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from helper_database import extract_store_listings_huurstunt

import logging

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
load_dotenv()

HUURSTUNT_USERNAME = os.getenv('HUURSTUNT_USERNAME')
HUURSTUNT_PASSWORD = os.getenv('HUURSTUNT_PASSWORD')

DATABASE_NAME = "listings.db"


chrome_options = webdriver.ChromeOptions()
chrome_options.debugger_address = "127.0.0.1:9222"
driver = webdriver.Chrome(options=chrome_options)

In [3]:
driver.get("https://www.huurstunt.nl/")
set_filters_huurstunt(driver, 100, 1500, 'Amsterdam')
listings = extract_store_listings_huurstunt(driver)

In [3]:
listings = extract_store_listings_huurstunt(driver)
for listing in listings:
    listing_id = listing['id']  # assuming each listing is a dictionary with an 'id' key
    if should_send_email    (listing_id):
        print(f"Email hasn't been sent for listing ID: {listing_id}")
    else:
        print(f"Email has already been sent for listing ID: {listing_id}")


Email hasn't been sent for listing ID: 1
Email hasn't been sent for listing ID: 2
Email hasn't been sent for listing ID: 3
Email hasn't been sent for listing ID: 4
Email hasn't been sent for listing ID: 6
Email hasn't been sent for listing ID: 7
Email hasn't been sent for listing ID: 8
Email hasn't been sent for listing ID: 10
Email hasn't been sent for listing ID: 11
Email hasn't been sent for listing ID: 12
Email hasn't been sent for listing ID: 13
Email hasn't been sent for listing ID: 15
Email hasn't been sent for listing ID: 16
Email hasn't been sent for listing ID: 17
Email hasn't been sent for listing ID: 18
Email hasn't been sent for listing ID: 19
Email hasn't been sent for listing ID: 20


In [9]:
# Step 1: Extract listings
try:
    listings = extract_store_listings_huurstunt(driver)
    logging.info(f'Extracted {len(listings)} listings.')
except Exception as e:
    logging.error(f"Error during listing extraction: {e}")

# Step 2: Process each listing
for listing in listings:
    listing_id = listing['id']  # Use the 'id' from the extracted data
    
    # Check if an email should be sent for this listing
    try:
        if should_send_email(listing_id):
            
            # Navigate to the specific URL of the listing
            driver.get(listing['specific_url'])
            
            # Extract owner email from the XPath (assuming `email_xpath` is the XPath for the email)
            email_element = driver.find_element(By.XPATH, email_xpath)
            owner_email = email_element.text
            
            # Send the notification email
            send_notification_to_owner(listing, address_selector, email_selector, listing['specific_url'])
            
            # Mark the email as sent in the database
            mark_email_as_sent(listing_id)
    except Exception as e:
        logging.error(f"Error processing listing with ID {listing_id}: {e}")


2023-10-31 23:07:44,264 - INFO - Extracted 17 listings.
2023-10-31 23:07:44,876 - ERROR - Error processing listing with ID 1: name 'email_xpath' is not defined
2023-10-31 23:07:45,515 - ERROR - Error processing listing with ID 2: name 'email_xpath' is not defined
2023-10-31 23:07:46,049 - ERROR - Error processing listing with ID 3: name 'email_xpath' is not defined
2023-10-31 23:07:46,698 - ERROR - Error processing listing with ID 4: name 'email_xpath' is not defined
2023-10-31 23:07:47,343 - ERROR - Error processing listing with ID 6: name 'email_xpath' is not defined
2023-10-31 23:07:47,867 - ERROR - Error processing listing with ID 7: name 'email_xpath' is not defined
2023-10-31 23:07:48,819 - ERROR - Error processing listing with ID 8: name 'email_xpath' is not defined
2023-10-31 23:07:49,791 - ERROR - Error processing listing with ID 10: name 'email_xpath' is not defined
2023-10-31 23:07:50,602 - ERROR - Error processing listing with ID 11: name 'email_xpath' is not defined
2023-1

In [28]:
# Step 1: Extract listings
#driver.get("https://www.huurstunt.nl/studio/huren/amsterdam/100-1500/")
try:
    listings = extract_store_listings_huurstunt(driver)
    logging.info(f'Extracted {len(listings)} listings.')
except Exception as e:
    logging.error(f"Error during listing extraction: {e}")

# Step 2: Process each listing
for listing in listings[:4]:
    listing_id = listing['id']  # Use the 'id' from the extracted data
    logging.info(f"Processing listing with ID {listing_id}")
    
    # Check if an email should be sent for this listing
    try:
        if should_send_email(listing_id):
            logging.info(f"Email hasn't been sent for listing ID: {listing_id}")
            
            # Navigate to the specific URL of the listing
            driver.get(listing['specific_url'])
            print('got the specific url: ', listing['specific_url'])
            
            # Extract owner email from the XPath (assuming `email_xpath` is the XPath for the email)
            email_element = driver.find_element(By.XPATH, "//a[contains(@href, 'mailto:')]/span[@data-select-copy='1']")
            owner_email = email_element.text

            # Print the extracted email for verification
            print(f"Extracted Email for {listing['address']}: {owner_email}")
            
    except NoSuchElementException:
        logging.error(f"Unable to locate email for listing with ID {listing_id}")
        # Send a notification to your OUTLOOK_EMAIL
        print('MAKE THE FUNCTION TO SEND THE EMAIL TO YOURSELF')
        # send_notification_to_my_email(listing['specific_url'])  # Assuming you have a method to send an email to your address
    except Exception as e:
        logging.error(f"Error processing listing with ID {listing_id}: {e}")

            
    # Commented out the actual sending of the email. Uncomment this when you're ready to send.
    # """
    # # Send the notification email
    # send_notification_to_owner(listing, address_selector, email_selector, listing['specific_url'])
    
    # # Mark the email as sent in the database
    # mark_email_as_sent(listing_id)
    #         """


2023-10-31 23:26:53,499 - INFO - Extracted 19 listings.
2023-10-31 23:26:53,499 - INFO - Processing listing with ID 1
2023-10-31 23:26:53,506 - INFO - Email hasn't been sent for listing ID: 1


got the specific url:  https://www.huurstunt.nl/kamer/huren/in/amsterdam/wijttenbachstraat/Q1FYm


NameError: name 'NoSuchElementException' is not defined

In [26]:
for listing in listings[:4]:
    listing_id = listing['id']
    try:
        email = driver.execute_script('return document.querySelector("span[data-select-copy=\'1\']").innerText;')
        # email = email_element.text
        print(f"Extracted Email for {listing_id}: {email}")
    except NoSuchElementException:
        print(f"Error: Couldn't extract email for {listing_id}")

Extracted Email for 1: info@hendrikshousing.nl
Extracted Email for 2: info@hendrikshousing.nl
Extracted Email for 3: info@hendrikshousing.nl
Extracted Email for 4: info@hendrikshousing.nl


In [24]:
email = driver.execute_script('return document.querySelector("span[data-select-copy=\'1\']").innerText;')
email


'info@hendrikshousing.nl'

In [ ]:
import smtplib
from email.message import EmailMessage

def send_notification_email():
    msg = EmailMessage()
    msg.set_content("The email element was not found on the page.")
    msg["Subject"] = "Email Extraction Error!"
    msg["From"] = "your_email_address"
    msg["To"] = "your_outlook_email"

    # Establish a connection to your SMTP server
    with smtplib.SMTP("your_smtp_server", your_smtp_port) as server:
        server.login("your_email_address", "your_email_password")
        server.send_message(msg)
